<a href="https://colab.research.google.com/github/AMHlocal/final-project/blob/austin_branch/Grocery_ReviewsNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
#latest version of spark 3.0 from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

#Install spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

#Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd6

In [ ]:
#Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

In [ ]:
#Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cobrakaireviews.s3.us-east-2.amazonaws.com/amazon_reviews_us_Grocery_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv"), sep="\t", header=True)

#Show DF
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
# count how many rows before dropping nulls
df.count()

2402458

In [ ]:
# Find nulls in data
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|              24|         27|           27|         27|  27|               27|             29|        231|        180|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [ ]:
# Drop nulls in data
df1=df.dropna()
df1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [ ]:
# count number of rows after drop (should be less than 1741100(tools), 2402458(Grocery))
df1.count()

2402212

In [ ]:
#Concatenate headline and body columns
from pyspark.sql import functions as sf
df2 = df1.withColumn('joined', sf.concat(sf.col('review_headline'), sf.lit(' '), sf.col('review_body')))
df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|Using these for y...|
|         US|   12049833|R1OF8GP57AQ

### Feature Transformations


In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
tokenizer = Tokenizer(inputCol='joined', outputCol='tokened')
stopremove = StopWordsRemover(inputCol='tokened', outputCol='stopped')
hashing = HashingTF(inputCol='stopped', outputCol='hashed', numFeatures=pow(2,4))
IDF = IDF(inputCol='hashed', outputCol='idf')

In [ ]:
# tokenize review body
token = tokenizer.transform(df2)
token.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-0

In [ ]:
#stopwords
stopped = stopremove.transform(token)
stopped.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|    

In [ ]:
#hashing dataframe
hashed_df = hashing.transform(stopped)
hashed_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|              hashed|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...| 

In [ ]:
# fit to IDF
idfModel = IDF.fit(hashed_df)
scaledData = idfModel.transform(hashed_df)
scaledData.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|              joined|             tokened|             stopped|              hashed|                 idf|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         US|   42521656|

In [ ]:
 #drop non-useful columns
reviews = scaledData.drop('marketplace', 'customer_id', 'review_id', 'product_parent', 'product_title', 'product_category', 
                          'vine', 'verified_purchase', 'review_date', 'helpful_votes', 'total_votes')
reviews.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B000SAQCWC|          5|Using these for y...|As a family aller...|Using these for y...|[using, these, fo...|[using, years, -,...|(16,[0,1,3,4,5,6,...|(16,[0,1,3,4,5,6,...|
|B00509LVIQ|          5|           Wonderful|My favorite nut. ...|Wonderful My favo...|[wonderful, my, f...|[wonderful, favor...|(16,[0,1,2,3,4,6,...|(16,[0,1,2,3,4,6,...|
|B00KHXESLC|          5|          Five Stars|This green tea ta...|Five Stars This g...|[five, stars, thi...|[five, stars, gre...|(16,[1,2,6,

In [ ]:
# seeing if words are in dataframe and getting the count of the words
# common words: https://bit.ly/3mGZWEu, 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.functions import monotonically_increasing_id 


neg = ["broken", "broke", "brake", "dead", "died", "busted", "cracked", "ruin", "ruined", "failure", "failed", "chipped", "unstable", "defective", "shotty", "bent", "stop", "stopped", 		
	      "poor", "poorly", "risk", "risky", "bias", "bad", "sucks", "cheap", "tiny", "pain", "junk",
	      "replace", "rid", "back", "return", "tossed", "waste", "wasted", "highly", "too", "only",
       	"difficult", "adverse", "lack", "unclear", "negative", "wrong",
       	"hazard", "harm", "error", "severe", "exclusion", "hard", "absence", "loss", "falls",
	      "terrible", "disappointed", "disapointing", "horrible", "horribly", "ridiculously", "ridiculous", "inaccurate", "useless", "complex", "limitation", "limited", "confusing",
	      "junk", "crap", "shit", "shitty","sad",
	      "no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "hardly", "scarcely", "barely", "doesn’t", "isn’t", "wasn’t", "shouldn’t", "wouldn’t", "couldn’t", 
        "won’t", "can’t", "don’t", "doesnt", "didn't" ,"isnt", "wasnt", "shouldnt", "wouldnt", "couldnt", "wont", "cant", "dont", "didnt",
        "quit", "issue", "issues", "concerns", "concerned", "concern", "limitations", "problem", "problems", "hassle", "bother"]

def check(list):
  counter = 0
  for word in list:
    if word in neg:
      counter += 1
  return counter

def wordCount(list):
  return len(list)

def number(list):
  return int(list)

def ratio(list):
  return float(list)

pies = udf(lambda w : check (w), IntegerType())
apples = udf(lambda t : wordCount (t), IntegerType())
peach = udf(lambda e : number (e), IntegerType())
pear = udf(lambda f : ratio (f), FloatType())
zebra = reviews.withColumn('neg_count', pies(col('tokened')))
zebra = zebra.select("*").withColumn("id", monotonically_increasing_id())
zebra = zebra.select("*").withColumn("total_word_count", apples(col('tokened')))
zebra = zebra.select("*").withColumn("star_rating", peach(col('star_rating')))
zebra = zebra.select("*").withColumn("ratio", pear(col('neg_count')/col('total_word_count')))
# zebra = zebra[zebra['total_word_count'] > 10]
zebra.show()


+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+------------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count| id|total_word_count|       ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+------------+
|B000SAQCWC|          5|Using these for y...|As a family aller...|Using these for y...|[using, these, fo...|[using, years, -,...|(16,[0,1,3,4,5,6,...|(16,[0,1,3,4,5,6,...|        0|  0|              91|         0.0|
|B00509LVIQ|          5|           Wonderful|My favorite nut. ...|Wonderful My favo...|[wonderful, my, f...|[wonderful, favor...|(16,[0,

In [ ]:
# smaller dataframe to work with
(panda, fish) = zebra.randomSplit([0.2, 0.8])
panda.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+----------------+-----------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count|    id|total_word_count|      ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+------+----------------+-----------+
|0268956898|          5|You will lose wei...|This coffee has b...|You will lose wei...|[you, will, lose,...|[lose, weight, fe...|(16,[0,1,4,5,6,7,...|(16,[0,1,4,5,6,7,...|        0|116630|              37|        0.0|
|3334353648|          5|     Fresh and Good.|Good Fresh ginger...|Fresh and Good. G...|[fresh, and, good...|[fresh, good., go...

In [ ]:
dogs = zebra.filter((zebra.id > 0) & (zebra.id < 2402212))
dogs.show()

+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+------------+
|product_id|star_rating|     review_headline|         review_body|              joined|             tokened|             stopped|              hashed|                 idf|neg_count| id|total_word_count|       ratio|
+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---+----------------+------------+
|B00509LVIQ|          5|           Wonderful|My favorite nut. ...|Wonderful My favo...|[wonderful, my, f...|[wonderful, favor...|(16,[0,1,2,3,4,6,...|(16,[0,1,2,3,4,6,...|        0|  1|              18|         0.0|
|B00KHXESLC|          5|          Five Stars|This green tea ta...|Five Stars This g...|[five, stars, thi...|[five, stars, gre...|(16,[1,

In [ ]:
features = dogs.drop("product_id", "star_rating", "review_headline", "review_body", "joined", "tokened", "stopped", "hashed", "id", "total_word_count")
features.show()

+--------------------+---------+------------+
|                 idf|neg_count|       ratio|
+--------------------+---------+------------+
|(16,[0,1,2,3,4,6,...|        0|         0.0|
|(16,[1,2,6,7,13],...|        0|         0.0|
|(16,[0,1,2,6,8,14...|        0|         0.0|
|(16,[2,6,8],[0.27...|        0|         0.0|
|(16,[3,14,15],[0....|        1|  0.16666667|
|(16,[0,1,2,4,5,6,...|        0|         0.0|
|(16,[0,2,4,6,7,10...|        1|  0.04761905|
|(16,[0,1,2,4,5,6,...|        0|         0.0|
|(16,[0,1,2,3,5,6,...|        4|  0.06666667|
|(16,[0,1,2,3,4,5,...|        0|         0.0|
|(16,[2,6,7,8,11,1...|        0|         0.0|
|(16,[0,2,6,13,14]...|        0|         0.0|
|(16,[2,5,6,7,9,12...|        0|         0.0|
|(16,[2,4,6,8,9,14...|        0|         0.0|
|(16,[4,5,9,13,14]...|        0|         0.0|
|(16,[0,1,3,4,6,8,...|        1|  0.04761905|
|(16,[0,1,2,3,4,5,...|        1|0.0056497175|
|(16,[0,1,2,3,5,6,...|        1| 0.014925373|
|(16,[0,1,2,3,4,5,...|        0|  

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [ ]:
output = assembler.transform(dogs).select('features','star_rating')

In [ ]:
train,test = output.randomSplit([0.75, 0.25])

In [ ]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(featuresCol='features', labelCol="star_rating")
print(type(dogs))
fitted_model = model.fit(train)
print("Coefficients: " + str(fitted_model.coefficients))
print("Intercept: " + str(fitted_model.intercept))

<class 'pyspark.sql.dataframe.DataFrame'>
Coefficients: [0.08249260103598646,0.0010068700690373778,-0.2087658296108812,-0.007323467606618105,-0.035910211690995814,-0.014540547422961573,0.31303633674342757,-0.040959062542633415,-0.0063829667557174325,-0.00491461188698809,-0.0314192168194487,0.036067912708002844,-0.013221706364462916,0.07747153767200315,0.14627848224121662,-0.09899722951567654,-0.20692175678210184,-6.700768367785838]
Intercept: 4.566397288457888


In [ ]:
trainSummary = fitted_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 1.075899

r2: 0.183877


In [ ]:
from pyspark.sql.functions import abs
predictions = fitted_model.transform(test)
x = ((predictions['star_rating']- predictions['prediction'])/predictions['star_rating'])*100
predictions = predictions.withColumn('Accuracy', abs(x))
predictions.select("prediction","star_rating","Accuracy","features").show()

+-----------------+-----------+------------------+--------------------+
|       prediction|star_rating|          Accuracy|            features|
+-----------------+-----------+------------------+--------------------+
|4.632209248968409|          1| 363.2209248968409|(18,[0],[0.797792...|
|4.632209248968409|          1| 363.2209248968409|(18,[0],[0.797792...|
|4.632209248968409|          3| 54.40697496561363|(18,[0],[0.797792...|
|4.632209248968409|          4|15.805231224210225|(18,[0],[0.797792...|
|4.632209248968409|          4|15.805231224210225|(18,[0],[0.797792...|
|4.632209248968409|          4|15.805231224210225|(18,[0],[0.797792...|
|4.632209248968409|          5|7.3558150206318205|(18,[0],[0.797792...|
|4.632209248968409|          5|7.3558150206318205|(18,[0],[0.797792...|
|4.632209248968409|          5|7.3558150206318205|(18,[0],[0.797792...|
|4.632209248968409|          5|7.3558150206318205|(18,[0],[0.797792...|
|4.632209248968409|          5|7.3558150206318205|(18,[0],[0.797

In [ ]:
#r square for test dataset
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="star_rating",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.185658


In [ ]:
#adjusted r2
r2 = trainSummary.r2
n = dogs.count()
p = len(dogs.columns)
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.18385278742252487

In [ ]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='star_rating',maxIter=50, regParam=0.12, elasticNetParam=0.2)
linear_model = lin_reg.fit(train)